# Crypto 2 - POTP

> Fancier OTP service
> 
> - mechfrog88
>
> Files: [main.sage](main.sage) and [output.txt](output.txt)

So, the premise of this challenge is very short. You basically work in the polynomial ring $\mathbb{Z}_{8192}[x]$. You have three "text" polynomials $m_1, m_2, m_3$ whose coefficients are between 20 and 126 inclusive, and two random "byte" polynomials $o_2, o_3$ whose coefficients are between 0 and 255 inclusive. We also define $o_1 = o_2 + o_3$ to a be a third polynomial of a similar form, but this can take coefficients of between 0 and 510 inclusive.

Finally, we are given all three values of $m_i o_i$, for $i = 1,2,3$, and the flag is just the coefficients of $m_1$.

For the first hour of the competition I focused on Hensel-lifting the solution through increasing powers of 2. This probably would have worked, but at that point I (correctly or not) decided it would be easier to lift it through increasing powers of X in the polynomial instead. This works, and is in fact the intended solution, so I will discuss this first.

## Solution 1: Byte-by-byte through increasing powers of $X$

First, let's look at the polynomials we have been given. They are degree 126, which indicates that our $m_i$ and $o_i$ are all degree 63. We also print out the first six coefficients to see what they look like.

In [1]:
F.<x> = Zmod(2^13)[]
mo1, mo2, mo3 = [sage_eval(line, locals={'x':x}) for line in open('output.txt').readlines()]
print([mo.degree() for mo in [mo1,mo2,mo3]])
print(mo1[:6])
print(mo2[:6])
print(mo3[:6])

[126, 126, 126]
3698*x^5 + 1459*x^4 + 7995*x^3 + 1397*x^2 + 2480*x + 1325
5328*x^5 + 2865*x^4 + 6069*x^3 + 1091*x^2 + 646*x + 2768
6913*x^5 + 2380*x^4 + 1731*x^3 + 3230*x^2 + 3944*x + 7808


Ok, so let's focus only on the constant terms at first, i.e. $1325, 2768, 7808$. If we go through all possible constant-term solutions of $m_i$ and $o_i$, how many solutions are we expected to have? If it's less than one, then this means that our solution space will not grow exponentially, which is good!

First, let's count the number of possible inputs: we have 107 values between 20 and 126 inclusive for the $m_i$, and 256 bytes for the $o_i$. Put together, this gives us $107^3 \times 256^2 \approx 0.146 \times 8192^3$, or roughly 0.146 inputs for any given output, which is less than one. This means we can essentially brute-force all $107^3 \times 256^2$ at every step, rarely expect more than one solution, but if there are multiple solutions we expect the branch to die quickly in the next round or two. So let's code this up!

In [2]:
def test(c0,c1,c2,c3,c4,c5):
    
    print(bytes(c5))

    dim = len(c0)+1

    arr1 = []
    for a in range(256):
        for b in range(20,127):
            if (F(c0+[a])*F(c1+[b])-mo2)[:dim] == 0:
                arr1.append((a,b))

    arr2 = []
    for a in range(256):
        for b in range(20,127):
            if (F(c2+[a])*F(c3+[b])-mo3)[:dim] == 0:
                arr2.append((a,b))

    for a,b in arr1:
        for c,d in arr2:
            s = a+c
            for t in range(20,127):
                if (F(c4+[s])*F(c5+[t])-mo1)[:dim] == 0:
                    test(c0+[a],c1+[b],c2+[c],c3+[d],c4+[s],c5+[t])
                
test([],[],[],[],[],[])

b''
b'g'
b'g7'
b'gr'
b'gre'
b'grey'
b'grey@'
b'grey{'
b'grey{h'
b'grey{hF'
b'grey{hF\x18'
b'grey{hFS'
b'grey{hFS*'
b'grey{hFS*\x14'
b'grey{hFS*O'
b'grey{hFSe'
b'grey{hFSeQ'
b'grey{hFSeQZ'
b'grey{hFSeQZ7'
b'grey{hFSeQZr'
b'grey{hFSeQZr6'
b'grey{hFSeQZr6\x15'
b'grey{hFSeQZr6P'
b'grey{hFSeQZr6PP'
b'grey{hFSeQZr6PP3'
b'grey{hFSeQZr6PP3L'
b'grey{hFSeQZr6PP3L9'
b'grey{hFSeQZr6PP3L9o'
b'grey{hFSeQZr6PP3L9C'
b'grey{hFSeQZr6PP3L9C<'
b'grey{hFSeQZr6PP3L9Cw'
b'grey{hFSeQZr6PP3L9Cwx'
b'grey{hFSeQZr6PP3L9Cwx_'
b'grey{hFSeQZr6PP3L9Cwx_m'
b'grey{hFSeQZr6PP3L9Cwx_mo'
b'grey{hFSeQZr6PP3L9Cwx_mo7'
b'grey{hFSeQZr6PP3L9Cwx_mo7('
b'grey{hFSeQZr6PP3L9Cwx_mor'
b'grey{hFSeQZr6PP3L9Cwx_more'
b'grey{hFSeQZr6PP3L9Cwx_more_'
b'grey{hFSeQZr6PP3L9Cwx_more_v'
b'grey{hFSeQZr6PP3L9Cwx_more_vu'
b'grey{hFSeQZr6PP3L9Cwx_more_vu1'
b'grey{hFSeQZr6PP3L9Cwx_more_vul'
b'grey{hFSeQZr6PP3L9Cwx_more_vul3'
b'grey{hFSeQZr6PP3L9Cwx_more_vuln'
b'grey{hFSeQZr6PP3L9Cwx_more_vuln*'
b'grey{hFSeQZr6PP3L9Cwx_more_vulne'
b'grey{hFSeQZr6PP3

There's a few branches, but the flag comes out quite clearly:

`grey{hFSeQZr6PP3L9Cwx_more_vulnerable_than_xor_f9xZ3tDhTMVT73bu}`

## Solution 2: Hensel lifting powers of 2

So the theory is that we can factorise `mo1` directly into two polynomials of degree 63. Well, we know we can definitely factorise it in GF(2), but Zmod(8192) is less clear. But let's see if we can do it increasing powers of 2.

Let's start with GF(2) first, where we can easily factorise it (e.g. using [Cantor-Zassenhaus](https://en.wikipedia.org/wiki/Cantor%E2%80%93Zassenhaus_algorithm)), and it turns out there are exactly two factors of degree 63. Moreover, we are fortunate enough that the leading coefficient is odd, so is a unit in Zmod(8192). This allows us to normalise the polynomial so that we only work with polynomials with leading coefficient 1.

In [3]:
norm1 = mo1 / mo1[126]

def all_factors(prod):
    return map(product, cartesian_product([[a^c for c in range(b+1)] for a,b in prod.factor()]))

f, g = [f for f in all_factors(mo1.change_ring(GF(2))) if f.degree() == 63]
f, g

(x^63 + x^62 + x^60 + x^59 + x^58 + x^56 + x^55 + x^54 + x^53 + x^52 + x^51 + x^50 + x^49 + x^48 + x^46 + x^45 + x^42 + x^41 + x^40 + x^39 + x^36 + x^34 + x^33 + x^30 + x^28 + x^27 + x^26 + x^22 + x^19 + x^18 + x^15 + x^13 + x^11 + x^10 + x^9 + x^8 + x^7 + x^6 + 1,
 x^63 + x^62 + x^60 + x^59 + x^56 + x^51 + x^48 + x^46 + x^44 + x^42 + x^40 + x^37 + x^36 + x^33 + x^31 + x^28 + x^26 + x^25 + x^23 + x^22 + x^21 + x^19 + x^18 + x^17 + x^15 + x^9 + x^8 + x^7 + x^4 + x^3 + x^2 + 1)

So, what happens here is that we represent out normalised target polynomial as a product of two degree-63 polynomials $p=fg$. We were able to factorise it in GF(2), so we know the values of $f \bmod{2}$ and $g \bmod{2}$, as above. Now, as we increase the modulus to higher powers of 2, we still have the leading coefficients of $f$ and $g$ be one, so we only need to add degree-62 polynomials, i.e.

$$p = (f + 2\delta_f)(g + 2\delta_g) \pmod{4}.$$

Which means we are trying to find binary polynomials $\delta_f$ and $\delta_g$ such that

$$p-fg = 2(f \delta_g + g \delta_f),$$

and in this way we can iteratively learn the lower bits of $f$ and $g$.

In [4]:
target = ((norm1.change_ring(ZZ) - f.change_ring(ZZ)*g.change_ring(ZZ))/2).change_ring(GF(2))
target

x^125 + x^121 + x^119 + x^115 + x^112 + x^110 + x^109 + x^108 + x^107 + x^101 + x^100 + x^99 + x^98 + x^97 + x^96 + x^93 + x^89 + x^84 + x^83 + x^82 + x^81 + x^80 + x^79 + x^78 + x^73 + x^70 + x^69 + x^68 + x^67 + x^65 + x^64 + x^59 + x^58 + x^57 + x^55 + x^54 + x^53 + x^50 + x^48 + x^45 + x^43 + x^42 + x^40 + x^39 + x^38 + x^37 + x^34 + x^33 + x^32 + x^30 + x^29 + x^26 + x^23 + x^21 + x^20 + x^19 + x^17 + x^15 + x^13 + x^12 + x^10 + x^9 + x^7 + x^5 + x^2 + 1

In [5]:
_, finv, ginv = xgcd(f, g)
df = target * ginv % f
dg = target * finv % g
f * dg + g * df == target

True

In [6]:
R = Zmod(4)
f2 = f.change_ring(R)+df.change_ring(R)*2
g2 = g.change_ring(R)+dg.change_ring(R)*2
f2 * g2 == norm1 # mod 4

True

In [7]:
target2 = ((norm1.change_ring(ZZ) - f2.change_ring(ZZ)*g2.change_ring(ZZ))/4).change_ring(GF(2))
target2

x^124 + x^123 + x^121 + x^120 + x^119 + x^118 + x^114 + x^113 + x^112 + x^111 + x^110 + x^109 + x^107 + x^106 + x^105 + x^104 + x^103 + x^101 + x^100 + x^99 + x^96 + x^89 + x^84 + x^83 + x^73 + x^71 + x^70 + x^68 + x^66 + x^65 + x^64 + x^63 + x^61 + x^59 + x^58 + x^57 + x^55 + x^54 + x^53 + x^51 + x^50 + x^49 + x^48 + x^47 + x^46 + x^41 + x^39 + x^37 + x^36 + x^33 + x^32 + x^30 + x^28 + x^25 + x^18 + x^17 + x^15 + x^14 + x^12 + x^11 + x^9 + x^8 + x^7 + x^6 + x^3 + x^2 + 1

At this point I had the lower 2 bits of $f$ and $g$, but I couldn't get the maths to work properly in lifting from $\bmod{4}$ to $\bmod{8}$. So I gave up here, and got the flag via Solution 1.

### However!

After the CTF ended I was informed by @4yn that sage 10 can already factorise it natively. See this for yourself!

Copy the following into [SageMathCell](https://sagecell.sagemath.org/), which presumably runs the latest sage:
```py
F.<x> = Zmod(2^13)[]
mo1 = 491*x^126 + 5268*x^125 + 771*x^124 + 890*x^123 + 4024*x^122 + 7391*x^121 + 3590*x^120 + 2674*x^119 + 111*x^118 + 7817*x^117 + 6268*x^116 + 5653*x^115 + 7670*x^114 + 3655*x^113 + 1089*x^112 + 6273*x^111 + 5160*x^110 + 5060*x^109 + 3763*x^108 + 3620*x^107 + 2338*x^106 + 6273*x^105 + 6775*x^104 + 6846*x^103 + 1858*x^102 + 6694*x^101 + 8177*x^100 + 3416*x^99 + 1909*x^98 + 1622*x^97 + 7381*x^96 + 7834*x^95 + 6464*x^94 + 5063*x^93 + 2805*x^92 + 4230*x^91 + 7883*x^90 + 3346*x^89 + 4100*x^88 + 832*x^87 + 5107*x^86 + 502*x^85 + 7720*x^84 + 2352*x^83 + 3470*x^82 + 3100*x^81 + 6762*x^80 + 3256*x^79 + 7799*x^78 + 795*x^77 + 3928*x^76 + 7325*x^75 + 7747*x^74 + 3480*x^73 + 6675*x^72 + 6678*x^71 + 4996*x^70 + 2948*x^69 + 822*x^68 + 2997*x^67 + 1621*x^66 + 3023*x^65 + 5612*x^64 + 7353*x^63 + 1765*x^62 + 5443*x^61 + 1833*x^60 + 6925*x^59 + 7112*x^58 + 7141*x^57 + 2964*x^56 + 8086*x^55 + 1423*x^54 + 1786*x^53 + 6053*x^52 + 6808*x^51 + 3524*x^50 + 6585*x^49 + 2962*x^48 + 3761*x^47 + 6192*x^46 + 7762*x^45 + 6530*x^44 + 601*x^43 + 7095*x^42 + 1012*x^41 + 3688*x^40 + 2057*x^39 + 1598*x^38 + 2058*x^37 + 6615*x^36 + 7527*x^35 + 4087*x^34 + 7888*x^33 + 2405*x^32 + 6291*x^31 + 6871*x^30 + 3274*x^29 + 4438*x^28 + 5637*x^27 + 3565*x^26 + 3629*x^25 + 991*x^24 + 6909*x^23 + 4277*x^22 + 8145*x^21 + 1579*x^20 + 6905*x^19 + 5654*x^18 + 1040*x^17 + 812*x^16 + 6387*x^15 + 6116*x^14 + 6317*x^13 + 2423*x^12 + 5768*x^11 + 1166*x^10 + 2450*x^9 + 2673*x^8 + 1892*x^7 + 4923*x^6 + 3698*x^5 + 1459*x^4 + 7995*x^3 + 1397*x^2 + 2480*x + 1325

def all_factors(prod):
    return map(product, cartesian_product([[a^c for c in range(b+1)] for a,b in prod.factor()]))

for f in all_factors(mo1):
    if f.degree() == 63:
        print(''.join(chr(x*ord('}')%256) for x in f))
```

Which basically finds all factors of degree 63 and prints it as a string (assuming it ends with a `}`).

This prints the flag `grey{hFSeQZr6PP3L9Cwx_more_vulnerable_than_xor_f9xZ3tDhTMVT73bu}`.